# Weak-lensing galaxy shape catalogue validation

## Maps

Contents
- Create convergence maps with local calibration

> **_NOTE:_** Before running this notebook, set kernel to `main_set.ipynb'

In [ ]:
from scipy.ndimage.filters import gaussian_filter

from lenspack.utils import bin2d
from lenspack.image.inversion import ks93

In [ ]:
from sp_validation.survey import *
from sp_validation.util import *
from sp_validation.basic import *
from sp_validation.plots import *
from sp_validation.cosmology import *

# Project to Cartesian coordinates

In [ ]:
x_local = {}
y_local = {}

for sh in shapes:

    x_local[sh] = {}
    y_local[sh] = {}
    
    for cal_pix in cal_pix_size_deg:

        ra_mean_local = np.mean(ra_local[sh][cal_pix])
        dec_mean_local = np.mean(dec_local[sh][cal_pix])
        x_local[sh][cal_pix], y_local[sh][cal_pix] = radec2xy(
            ra_mean_local,
            dec_mean_local,
            ra_local[sh][cal_pix],
            dec_local[sh][cal_pix]
        )

# Bin in 2D

In [ ]:
g_corr_local_m_global_c_map = {}

for sh in shapes:

    g_corr_local_m_global_c_map[sh] = {}
    
    for cal_pix in cal_pix_size_deg:
        g1_tmp, g2_tmp = bin2d(
            x_local[sh][cal_pix],
            y_local[sh][cal_pix],
            npix=(Nx[sh], Ny[sh]), 
            v=(
                g_corr_local_m_global_c[sh][cal_pix][0],
                g_corr_local_m_global_c[sh][cal_pix][1]
            ),
            extent=(min_x[sh], max_x_new[sh], min_y[sh], max_y_new[sh])
        )

        g_corr_local_m_global_c_map[sh][cal_pix] = np.array([g1_tmp, g2_tmp])

# Create convergence maps

In [ ]:
# Transform gamma -> kappa using the Kaiser-Squires (1993) algorithm

kappaE_local = {}
kappaB_local = {}

for sh in shapes:
    
    kappaE_local[sh] = {}
    kappaB_local[sh] = {}
    
    for cal_pix in cal_pix_size_deg:
        
        kappaE_local[sh][cal_pix], kappaB_local[sh][cal_pix] = ks93(
            g1_sign * g_corr_local_m_global_c_map[sh][cal_pix][0],
            g2_sign * g_corr_local_m_global_c_map[sh][cal_pix][1]
        )

In [ ]:
# Smooth with Gaussian filter

kappaE_sm_local = {}
kappaB_sm_local = {}

for sh in shapes:
    
    kappaE_sm_local[sh] = {}
    kappaB_sm_local[sh] = {}
    
    for cal_pix in cal_pix_size_deg:

        kappaE_sm_local[sh][cal_pix] = gaussian_filter(kappaE_local[sh][cal_pix], smoothing_scale_pix)
        kappaB_sm_local[sh][cal_pix] = gaussian_filter(kappaB_local[sh][cal_pix], smoothing_scale_pix)

## Get known cluster positions

In [ ]:
# Get cluster information
cluster_cat_name = 'HFI_PCCS_SZ-union_R2.08.fits.gz'
vos_dir = 'vos:cfis/cosmostat/cosmology/external/Planck'

clusters = get_clusters(cluster_cat_name, vos_dir, data_dir, name, verbose=verbose)

print_stats(
    f"{len(clusters['ra'])} clusters found in {name} footprint",
    stats_file,
    verbose=verbose
)

In [ ]:
# Project cluster positions

# Get first shape method, use the same for all projections
for sh in shapes:
    sh0 = sh
    break

x_cluster, y_cluster =  radec2xy(
    ra_mean[sh0],
    dec_mean[sh0],
    clusters['ra'],
    clusters['dec']
)
clusters['x'] = x_cluster
clusters['y'] = y_cluster

# Plot maps

## Stacked convergence maps

In [ ]:
# Comoving separation around cluster centers, at cluster redshift, in [Mpc]
radius = 5

res_stack_mm_local = {}

# Stack galaxies
for sh in shapes:

    res_stack_mm_local[sh] = {}

    for cal_pix in cal_pix_size_deg:
        
        res_stack_mm_local[sh][cal_pix] = stack_mm3(
            ra_local[sh][cal_pix],
            dec_local[sh][cal_pix],
            g_corr_local_m_global_c[sh][cal_pix][0],
            g_corr_local_m_global_c[sh][cal_pix][1],
            w[sh],
            clusters['ra'],
            clusters['dec'],
            clusters['z'],
            radius=radius,
            n_match=1000000
        )

In [ ]:
# Plot stacked galaxy density, to check how uniform distribution is. Sometimes at the edges the number
# of galaxies drops visibly

for sh in shapes:
    
    plt.figure(figsize=(10 * n_cal, 10))

    for idx, cal_pix in enumerate(cal_pix_size_deg):

        plt.subplot(1, n_cal, idx+1)
        plt.hexbin(
            res_stack_mm_local[sh][cal_pix][0],
            res_stack_mm_local[sh][cal_pix][1],
            gridsize=100,
            cmap='gist_stern'
        )
        cbar = plt.colorbar()
        cbar.set_label('Number count', rotation=270)
        plt.title(f'{sh} stacked galaxy density plot {cal_pix}')

In [ ]:
# Plot stacked galaxy density, to check how uniform distribution is. Sometimes at the edges the number
# of galaxies drops visibly

for sh in shapes:
    
    plt.figure(figsize=(10 * n_cal, 10))

    for idx, cal_pix in enumerate(cal_pix_size_deg):

        sub_plot(
            1,
            n_cal,
            idx + 1,
            res_stack_mm_local[sh][cal_pix],
            rf'{sh}: galaxy density, local cal {cal_pix} deg'
        )